<h1 align=center><font size = 5>Business Analysis for opening a restaurant in Manhattan of New York City</font></h1>

## Introduction

A UK restaurants chain plans to open a new restaurant in Manhattan in New York City, since it is the most densely populated borough of New York City. Due to its
dense population there is a huge increase in demand for restaurant business and is Ideal place to open a restaurant. The vendor does not have an idea of location that
would be optimal for opening a restaurant hence they have reached out to me.Can We suggest an optimal location for opening a new restaurant to the vendor by
doing data analysis on current Restaurants and Geographical factors?

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each postalCode</a>

4. <a href="#item4">Cluster postalCode</a>

</font>
</div>

Download all the dependencies and import all the liabraireas that we will need for this project.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: - 

## 1. Download and Explore Dataset

The dataset used for this project is the json data of New York City. This data is made available by IBM and it can be found here("https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json") and is ready to use. The data contains info about all the Borough, its neighborhood and their Latitude, Longitude info. 

This dataset is available for free on the web, below is the the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

In [ ]:
!wget -q -O 'newyorkCity_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

#### Load and explore the data

Next, let's load the data.

In [ ]:
with open('newyorkCity_data.json') as json_data:
    newyorkCity_data = json.load(json_data)

Let's take a quick look at the data.

In [ ]:
newyorkCity_data

Kindly note all the relevant data is in the **features** key, which contains a list of the neighborhoods. So, let's extract these data and store it in a new variable.

In [ ]:
neighborhoods_data = newyorkCity_data['features']
#newyork_data.head()

Let's go through first row of this data.

In [ ]:
neighborhoods_data[0]

### Tranform the data into a pandas dataframe

Now, lets transform this data of nested Python dictionaries into a *pandas* dataframe. 

Create the indented columns and assign it to a panadas dataframe.

In [ ]:
# define the pandas dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# Assign columns to a empty dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Loop through the python dictionary data and fill the dataframe one by one row.

In [ ]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Since our target is to open a restaurant in manhattan. Hence, we will slice the original dataframe and create a new dataframe of the Manhattan data.

In [ ]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

Find the geo co-ordinates of Manhattan city and draw a map for visualizing all neighbors in Manahattan 

In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

**Folium** is a great visualization library. we can zoom in and zoom out in the below map, and upon clicking on each circle mark name of the neighborhood apperas.

In [ ]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'YUWOO1JLRIVU0R40QZ05IODMDMDD54DX3DVH5SEXLSZKU5F4' # your Foursquare ID
CLIENT_SECRET = 'KEHYSKHETV4KGPOBLXX3WGGDNR4LGPFKY1Z3WS2PAJPQWZQL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
search_query = 'restaurant'
radius = 5000
print(search_query + ' .... OK!')

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
manhattan_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
manhattan_filtered['categories'] = manhattan_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
manhattan_filtered.columns = [column.split('.')[-1] for column in manhattan_filtered.columns]

manhattan_filtered.head()

#### Drop the irrelevent coulmns

In [ ]:
manhattan_new=manhattan_filtered.drop(['categories','address','cc','city','country','crossStreet','distance','formattedAddress','labeledLatLngs','neighborhood','state'], axis=1)
manhattan_new.head()

# lets understand the data

In [ ]:
manhattan_new.describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure();
#dataframe_filtered1.plot.box()
manhattan_new["lat"].plot.hist()

#data not normalized

In [ ]:
plt.figure();
manhattan_new["lng"].plot.hist()

#data not normalized

In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map

# add popular spots to the map as blue circle markers

for lat, lng, label in zip(manhattan_new['lat'], manhattan_new['lng'], manhattan_new['name']):
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False
        ).add_to(venues_map)


venues_map

In [ ]:
#manhattan_new.head()
manhattan_new.shape


Let's check how many venues were returned for each postalCode

In [ ]:

manhattan_new.groupby('postalCode').count()
#manhattan_new.groupby('postalCode').count().reset_index(name='counts')

## 3. Analyze Each Postal code

In [ ]:
manhattan_new_onehot = pd.get_dummies(manhattan_new[['name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_new_onehot['postalCode'] = manhattan_new['postalCode'] 

# move name column to the first column
fixed_columns = [manhattan_new_onehot.columns[-1]] + list(manhattan_new_onehot.columns[:-1])
manhattan_new_onehot = manhattan_new_onehot[fixed_columns]
manhattan_new_onehot.head()

In [ ]:
manhattan_new_onehot.shape

#### Next, let's group rows by postalcode and by taking the mean of the frequency of occurrence of each category

In [ ]:
manhattan_new_grouped = manhattan_new_onehot.groupby('postalCode').mean().reset_index()
manhattan_new_grouped

In [ ]:
manhattan_new_grouped.shape

#### Let's print each postal code along with the top 5 most common restaurant

In [ ]:
num_top_Restaurant = 5

for hood in manhattan_new_grouped['postalCode']:
    print("----"+hood+"----")
    temp = manhattan_new_grouped[manhattan_new_grouped['postalCode'] == hood].T.reset_index()
    temp.columns = ['Restaurant Name','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_Restaurant))
    print('\n')

In [ ]:
def return_most_common_Restaurant(row, num_top_Restaurant):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_Restaurant]

In [ ]:
num_top_Restaurant = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postalCode']
for ind in np.arange(num_top_Restaurant):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
manhattan_new_sorted = pd.DataFrame(columns=columns)
manhattan_new_sorted['postalCode'] = manhattan_new_grouped['postalCode']

for ind in np.arange(manhattan_new_grouped.shape[0]):
    manhattan_new_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_new_grouped.iloc[ind, :], num_top_venues)

manhattan_new_sorted

## 4. Cluster Neighborhoods

In [ ]:
kclusters = 5


dataframe_grouped_clustering = manhattan_new_grouped.drop('postalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dataframe_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels=kmeans.labels_[0:19]
labels

In [ ]:
dataframe_grouped_clustering["Labels"] = labels
dataframe_grouped_clustering.head(5)

In [ ]:
dataframe_grouped_clustering.groupby('Labels').mean()

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, postalCode, cluster  in zip(manhattan_new['lat'], manhattan_new['lng'],manhattan_new['postalCode'],kmeans.labels_):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_manhattan)  
    
map_clusters

### Hence we can conclude that the postalCode area which has moderate numbers of restaurants will be ideal to open a restaurant.

# *Thank You*